In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

# Database Operations

> This module has necessary DB-related operations (e.g., query execution, SQL query AST parsing etc.)

In [ ]:
#| default_exp dbops

In [ ]:
#| export
from claimdb.configuration import *
import time
import sqlite3
from pathlib import Path

## Executing SQLite Queries with Timeout

First, we define a function that executes a query with a timeout. If the query takes longer than the timeout, it will be aborted.  

In [ ]:
#| export

def sqlite_execute_with_timeout(database_path: Path, # Path to the Sqlite database
                                query:str,       # The Sqlite query to execute
                                timeout:int=30): # Timeout in seconds
    """ Runs a Sqlite query with a timeout. """
    start = time.time()

    def progress():
        # SQLite calls this regularly during execution
        if time.time() - start > timeout:
            return 1   # non-zero => interrupt query
        return 0

    conn = sqlite3.connect(database_path)

    conn.row_factory = sqlite3.Row
    conn.set_progress_handler(progress, 1000)

    try:
        cursor = conn.execute(query)
        rows = cursor.fetchall()
        return [dict(row) for row in rows]
    
    except sqlite3.OperationalError as e:
        if "interrupted" in str(e):
            return "timeout"
        return "error"
    
    except Exception as e:
        return "error"

    finally:
        if conn: conn.set_progress_handler(None, 0); conn.close()

In [ ]:
dbpath = config.bird_databases_dir / 'address' / 'address.sqlite'
query = """SELECT name FROM sqlite_master WHERE type='table' LIMIT 1;"""

In [ ]:
#| notest

print(sqlite_execute_with_timeout(dbpath, query))

[{'name': 'CBSA'}]


### *High-information* SQL queries

To ensure the benchmark focuses on claims that require non-trivial reasoning over relational data, we filter SQL queries according to their structural complexity. We define a function that returns whether a query qualifies as **high-information**. Queries that involve only simple point lookups (e.g., `WHERE id = 10`) do not provide enough semantic material for meaningful fact verification and are therefore excluded.

A query is considered **high-information** if it contains at least one of the following features:

1. **Aggregates**  
   The query uses aggregate functions such as `MAX`, `MIN`, `SUM`, `COUNT`, or `AVG`. Aggregations create quantitative summaries that support richer factual assertions.

2. **Grouping**  
   The query uses `GROUP BY` (and optionally `HAVING`). Grouped queries allow comparisons across categories and enable more complex derived facts.

3. **Ordering or Superlatives**  
   The query uses `ORDER BY`, `LIMIT`, `TOP`, or any structure that induces ranking or superlative behavior. These queries define relative comparisons (e.g., highest, lowest, top-k), which are central to non-trivial claim generation.

4. **Window Functions**  
   The query uses analytic/window functions such as `ROW_NUMBER`, `RANK`, `DENSE_RANK`, `PARTITION BY`, or similar constructs. Window functions express advanced relational reasoning that goes beyond simple selection or filtering.

5. **3 or more tables joined (i.e., 2 JOINs)** almost always signal nontrivial relational reasoning and multi-entity information flow. (we exclude 2-table joins are they are extremely common and often trivial -- e.g., students JOIN schools. They don’t necessarily add conceptual richness.)

Only queries satisfying at least one of these criteria are retained for benchmark construction. They yield complex, relationally grounded facts that are substantially more challenging for verification systems, resulting in a more discriminative and higher-quality evaluation set.

In [ ]:
#| export
import sqlglot

In [ ]:
tree = sqlglot.parse_one("""
SELECT * FROM c WHERE c.name = "Michael"
""")

list(tree.find_all(sqlglot.exp.AggFunc))

[]

In [ ]:
tree = sqlglot.parse_one("""
SELECT x, RANK() OVER (ORDER BY x) AS r FROM numbers;
""")

list(tree.find_all(sqlglot.exp.AggFunc))

[Rank()]

In [ ]:
tree = sqlglot.parse_one("""
SELECT * FROM a JOIN b ON a.id=b.id
""")

list(tree.find_all(sqlglot.exp.Join))

[Join(
   this=Table(
     this=Identifier(this=b, quoted=False)),
   on=EQ(
     this=Column(
       this=Identifier(this=id, quoted=False),
       table=Identifier(this=a, quoted=False)),
     expression=Column(
       this=Identifier(this=id, quoted=False),
       table=Identifier(this=b, quoted=False))))]

In [ ]:
tree = sqlglot.parse_one("""
SELECT * FROM a LEFT JOIN b ON a.id=b.id JOIN c ON 1
""")

len(list(tree.find_all(sqlglot.exp.Join)))

2

In [ ]:
tree = sqlglot.parse_one("""
SELECT * FROM a, b WHERE a.id = b.id
""")

len(list(tree.find_all(sqlglot.exp.Join)))

1

What type of expressions does `slqglog` mean by `AggFunc`?

In [ ]:
agg_functions = [
    cls.__name__ 
    for cls in sqlglot.exp.AggFunc.__subclasses__()
]
print(agg_functions)

['BitwiseAndAgg', 'BitwiseOrAgg', 'BitwiseXorAgg', 'BoolxorAgg', 'BitmapConstructAgg', 'BitmapOrAgg', 'ParameterizedAgg', 'ArgMax', 'ArgMin', 'ApproxTopK', 'ApproxTopKAccumulate', 'ApproxTopKCombine', 'ApproxTopSum', 'ApproxQuantiles', 'ApproxPercentileCombine', 'Minhash', 'MinhashCombine', 'ApproximateSimilarity', 'Grouping', 'GroupingId', 'AnonymousAggFunc', 'HashAgg', 'Hll', 'ApproxDistinct', 'ArrayAgg', 'ArrayUniqueAgg', 'AIAgg', 'AISummarizeAgg', 'ArrayConcatAgg', 'ArrayUnionAgg', 'Avg', 'AnyValue', 'Lag', 'Lead', 'First', 'Last', 'FirstValue', 'LastValue', 'NthValue', 'ObjectAgg', 'Count', 'CountIf', 'DenseRank', 'GroupConcat', 'JSONObjectAgg', 'JSONBObjectAgg', 'JSONArrayAgg', 'LogicalOr', 'LogicalAnd', 'Max', 'Median', 'Mode', 'Min', 'Ntile', 'PercentileCont', 'PercentileDisc', 'PercentRank', 'Quantile', 'ApproxPercentileAccumulate', 'Rank', 'RegrValx', 'RegrValy', 'RegrAvgy', 'RegrAvgx', 'RegrCount', 'RegrIntercept', 'RegrR2', 'RegrSxx', 'RegrSxy', 'RegrSyy', 'RegrSlope', 'Sum

In [ ]:
#| export

def is_query_high_information(query  # The Sqlite query to analyze
                              ) -> bool:  # Whether the query is high-information 
    """ Returns whether a query has high-info characteristics."""
    tree = sqlglot.parse_one(query, dialect='sqlite')

    num_joins = len(list(tree.find_all(sqlglot.exp.Join)))
    aggregates = list(tree.find_all(sqlglot.exp.AggFunc))
    orderings = list(tree.find_all(sqlglot.exp.Order))

    if aggregates or orderings or num_joins >= 2:
        return True
    return False

In [ ]:
is_query_high_information("SELECT COUNT(*) FROM table WHERE col = 'value';")

True

In [ ]:
is_query_high_information("SELECT * FROM table WHERE col = 'value';")

False

In [ ]:
is_query_high_information("SELECT * FROM a LEFT JOIN b ON a.id=b.id JOIN c ON 1;")

True

In [ ]:
is_query_high_information("SELECT * FROM a LEFT JOIN b ON a.id=b.id;")

False

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()